In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries
import time
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MultipleLocator

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

import nltk

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

data = pd.read_csv(r"..\..\data\covtype.csv")
train_df = pd.read_csv(r"..\..\data\train.csv")
test_df = pd.read_csv(r"..\..\data\test.csv")

### Feature Engineering
# Drop "ID"
train_df = train_df.drop(columns=["Id"])

# Create Labels DF
train_labels_df = train_df[["Cover_Type"]]

# Drop Labels
train_df = train_df.drop(columns=["Cover_Type"])

# Build the np arrays
train_data = train_df.to_numpy()
test_data = test_df.drop(columns=["Id"]).to_numpy()
train_labels = train_labels_df.to_numpy().ravel()

# Shuffle the input
shuffle = np.random.permutation(np.arange(train_data.shape[0]))
train_data, train_labels = train_data[shuffle], train_labels[shuffle]

# Set some variables to hold dev, and training data.
dev_data, dev_labels = train_data[14000:], train_labels[14000:]
train_data, train_labels = train_data[:14000], train_labels[:14000]

## Decision Tree

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn import tree

def print_importances(clf):
    importances = np.round(clf.feature_importances_,4)
    features = train_df.columns[0:55].to_numpy()
    importances_df = pd.DataFrame({'feature':features,'importance':importances})
    importances_df = importances_df.sort_values('importance',ascending=False)
    importances_df = importances_df[(importances_df.sum(axis=1) != 0)]  
    print(importances_df)
    
def print_structure(clf):
    n_nodes = clf.tree_.node_count
    children_left = clf.tree_.children_left
    children_right = clf.tree_.children_right
    feature = clf.tree_.feature
    threshold = clf.tree_.threshold

    node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, 0)]  # start with the root node id (0) and its depth (0)
    while len(stack) > 0:
        # `pop` ensures each node is only visited once
        node_id, depth = stack.pop()
        node_depth[node_id] = depth

        # If the left and right child of a node is not the same we have a split
        # node
        is_split_node = children_left[node_id] != children_right[node_id]
        # If a split node, append left and right children and depth to `stack`
        # so we can loop through them
        if is_split_node:
            stack.append((children_left[node_id], depth + 1))
            stack.append((children_right[node_id], depth + 1))
        else:
            is_leaves[node_id] = True

    print("The binary tree structure has {n} nodes and has the following tree structure:\n".format(n=n_nodes))
    for i in range(n_nodes):
        if is_leaves[i]:
            values = clf.tree_.value[i]
            max_value = values.max()
            cover_type = np.argmax(values)
            print("{space}node={node} is a leaf node with max value of {value} and cover type of {ctype}"
                  .format(space=node_depth[i] * "\t", node=i, value=max_value, ctype=cover_type))
        else:
            print("{space}node={node} is a split node: go to node {left} if {feature} <= {threshold} else to node {right}.".format(
                      space=node_depth[i] * "\t",
                      node=i,
                      left=children_left[i],
                      feature=train_df.columns[feature[i]],
                      threshold=threshold[i],
                      right=children_right[i]))



In [3]:
clf = tree.DecisionTreeClassifier(max_depth=4, random_state=0)
clf = clf.fit(train_data, train_labels)
predicted_labels = clf.predict(dev_data)
tree_model_score = clf.score(dev_data, dev_labels)
depth = clf.get_depth()
num_leaves = clf.get_n_leaves()

print('Desicion Tree Score: %.2f%%' % (tree_model_score*100))   
print('Decision Tree Depth is %d and has %d leaves' % (depth, num_leaves))

#plt.figure(figsize=(75,10))
#tree.plot_tree(clf, feature_names=train_df.columns, proportion=True)
#plt.show()
#print_importances(clf)
#print_structure(clf)
print("Confusion Matrix:\n")
print(confusion_matrix(dev_labels, predicted_labels))



Desicion Tree Score: 65.89%
Decision Tree Depth is 4 and has 16 leaves
Confusion Matrix:

[[ 97  23   0   0  15   0  30]
 [ 38  62   0   0  49   6   4]
 [  0   1  73  20   9  60   0]
 [  0   0   7 137   0   9   0]
 [  0  11   0   0 142  11   0]
 [  0   0  38  13  11  92   0]
 [ 27   0   0   0   0   0 135]]


In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

depths = {"max_depth": [3, 4, 5, 10, 25, 30]}

#Grid search is not working as expected.  Keeps claming the second to last is better even though it is not...
grid_search_decision_tree = GridSearchCV(tree.DecisionTreeClassifier(), depths, scoring='accuracy')
grid_search_decision_tree.fit(train_data, train_labels)
predicted = grid_search_decision_tree.predict(dev_data)
optimal_depth = grid_search_decision_tree.best_params_['max_depth']
print("The optimal value for depth using GridSearchCV method is {depth} with accuracy of {accuracy}"
      .format(depth=optimal_depth, accuracy=metrics.accuracy_score(dev_labels, predicted)))

clf_final = tree.DecisionTreeClassifier(max_depth=25, random_state=0)
clf_final = clf_final.fit(train_data, train_labels)
test_predictions = clf_final.predict(test_data)

predictions_dt = pd.DataFrame(data = test_predictions, index = test_df.loc[:, "Id"], columns = ["Cover_Type"])
predictions_dt.to_csv("dt_predictions.csv")  #Kaggle Score of 0.65796
predictions_dt 

The optimal value for depth using GridSearchCV method is 30 with accuracy of 0.8053571428571429


,Cover_Type
Id,
15121,2
15122,2
15123,1
15124,2
15125,2
...,...
581008,6
581009,6
581010,6


In [5]:
from sklearn.ensemble import AdaBoostClassifier

bdt = AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=20), 
                              n_estimators=50, learning_rate=1,
                              algorithm="SAMME")
bdt.fit(train_data, train_labels)
boosted_tree_model_score = bdt.score(dev_data, dev_labels)
print("Boosted Model Score: ", boosted_tree_model_score)

boosted_cv_score = cross_val_score(bdt, dev_data, dev_labels, cv = 10)
print("Boosted Decision Tree 10-Fold Cross Validation Scores:")
print(boosted_cv_score)
print("Mean: ", boosted_cv_score.mean())


boosted_predictions = bdt.predict(test_data)
predictions_bdt = pd.DataFrame(data = boosted_predictions, index = test_df.loc[:, "Id"], columns = ["Cover_Type"])
predictions_bdt.to_csv("bdt_predictions.csv")  #Kaggle Score of 0.74566 increase of 10%!!!


Boosted Model Score:  0.8741071428571429
Boosted Decision Tree 10-Fold Cross Validation Scores:
[0.6875     0.625      0.58928571 0.65178571 0.65178571 0.66071429
 0.70535714 0.57142857 0.66964286 0.65178571]
Mean:  0.6464285714285715


In [6]:
from sklearn.ensemble import RandomForestClassifier

rnd = RandomForestClassifier(criterion="entropy", max_depth=20, bootstrap=True, random_state=0)
rnd.fit(train_data, train_labels)
random_forest_score = rnd.score(dev_data, dev_labels)
print("Random Forest Score: " , random_forest_score)

random_forest_cv_score = cross_val_score(rnd, dev_data, dev_labels, cv = 10)
print("Random Forest 10-Fold Cross Validation Scores:")
print(random_forest_cv_score)
print("Mean: ", random_forest_cv_score.mean())

rf_predictions = rnd.predict(test_data)
predictions_rf = pd.DataFrame(data = rf_predictions, index = test_df.loc[:, "Id"], columns = ["Cover_Type"])
predictions_rf.to_csv("rf_predictions.csv")  #Kaggle Score of 0.73459


Random Forest Score:  0.8651785714285715
Random Forest 10-Fold Cross Validation Scores:
[0.73214286 0.76785714 0.75892857 0.69642857 0.70535714 0.76785714
 0.76785714 0.72321429 0.71428571 0.80357143]
Mean:  0.7437500000000001


In [7]:
### Warning - this runs VERY slow with n_estimators at 1000
from sklearn.ensemble import ExtraTreesClassifier
ex = ExtraTreesClassifier(random_state=0, n_estimators = 1000) 
ex.fit(train_data, train_labels)
extra_trees_score = ex.score(dev_data, dev_labels)
print("Extra Trees Score: ", extra_trees_score)

print("Random Forest 10-Fold Cross Validation Scores: ")
extra_trees_cv_score = cross_val_score(ex, dev_data, dev_labels, cv = 10)
print(extra_trees_cv_score)
print("Mean: ", extra_trees_cv_score.mean())

ex_predictions = ex.predict(test_data)
predictions_ex = pd.DataFrame(data = ex_predictions, index = test_df.loc[:, "Id"], columns = ["Cover_Type"])
predictions_ex.to_csv("ex_predictions.csv")  #Kaggle Score of 0.75287


Extra Trees Score:  0.8839285714285714
Random Forest 10-Fold Cross Validation Scores: 
[0.75       0.77678571 0.75892857 0.69642857 0.75       0.79464286
 0.75892857 0.76785714 0.78571429 0.80357143]
Mean:  0.7642857142857143


In [8]:
### Boosting with Random Forest
brf = AdaBoostClassifier(RandomForestClassifier(criterion="entropy", max_depth=20, bootstrap=True, random_state=0), 
                              n_estimators=50, learning_rate=1,
                              algorithm="SAMME")
brf.fit(train_data, train_labels)
boosted_forest_score = brf.score(dev_data, dev_labels)
print("Boosted Random Forest Score: ", boosted_forest_score)

boosted_forest_cv_score = cross_val_score(brf, dev_data, dev_labels, cv = 10)
print("Boosted Random Forest 10-Fold Cross Validation Scores:")
print(boosted_forest_cv_score)
print("Mean: ", boosted_forest_cv_score.mean())

boosted_forest_predictions = brf.predict(test_data)
predictions_brf = pd.DataFrame(data = boosted_predictions, index = test_df.loc[:, "Id"], columns = ["Cover_Type"])
predictions_brf.to_csv("brf_predictions.csv")  #Kaggle Score of 0.74566

Boosted Random Forest Score:  0.8705357142857143
Boosted Random Forest 10-Fold Cross Validation Scores:
[0.70535714 0.76785714 0.75       0.72321429 0.70535714 0.75892857
 0.75892857 0.75892857 0.74107143 0.8125    ]
Mean:  0.7482142857142857
